In [15]:
!pip install pygad
!pip install optuna

In [4]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pygad
import optuna
from statistics import median,mean
import random
from multiprocessing import Pool

In [5]:
base_graphs=[nx.to_numpy_array(nx.erdos_renyi_graph(random.randint(10,50),random.uniform(0.5, 0.9),directed=False)) for i in range(25)]

In [6]:
def obj(trial):
    params = {
        "sol_per_pop": trial.suggest_int("sol_per_pop", 50, 100),
        "num_parents_mating":2,
        "crossover_probability":trial.suggest_float("crossover_probability",0.8,0.9,log=False),
        "mutation_probability":trial.suggest_float("mutation_probability",0.01,0.05,log=False),
        "gene_space":[0,1],
        "parent_selection_type":trial.suggest_categorical("parent_selection_type", ["rws","tournament"]),
        "num_generations":100
    }
    num_iters=[]
    fit_vals=[]

    for graph in base_graphs:
       params["num_genes"]=len(graph[0])
       params["fitness_func"]= lambda solution, solution_idx: sum([solution[i]-len(solution)*solution[i]*sum([solution[j]*graph[i][j] for j in range(i,len(solution))]) for i in range(len(solution))])
       ga_instance = pygad.GA(**params)
       ga_instance.run()
       solution, solution_fitness, solution_idx = ga_instance.best_solution()
       num_iters.append(ga_instance.best_solution_generation)
       fit_vals.append(solution_fitness)

    niter=mean(num_iters)
    fval=mean(fit_vals)
    return niter,fval
study = optuna.create_study(directions=["minimize", "maximize"])
study.optimize(obj, n_trials=30)#, timeout=300

print("Number of finished trials: ", len(study.trials))

[I 2023-04-04 13:22:07,893] A new study created in memory with name: no-name-ecfdc583-d79c-43a2-bc5c-b48c8f5e41ef
[I 2023-04-04 13:24:02,292] Trial 0 finished with values: [33.0, 3.72] and parameters: {'sol_per_pop': 97, 'crossover_probability': 0.8796486874232897, 'mutation_probability': 0.014410223502521639, 'parent_selection_type': 'tournament'}. 
[I 2023-04-04 13:25:39,947] Trial 1 finished with values: [16.0, -971.8] and parameters: {'sol_per_pop': 84, 'crossover_probability': 0.817563282334348, 'mutation_probability': 0.011879386102601966, 'parent_selection_type': 'rws'}. 
[I 2023-04-04 13:27:36,424] Trial 2 finished with values: [36.0, -0.28] and parameters: {'sol_per_pop': 67, 'crossover_probability': 0.8285395390896183, 'mutation_probability': 0.03963303908446207, 'parent_selection_type': 'tournament'}. 
[I 2023-04-04 13:28:52,664] Trial 3 finished with values: [13.0, -886.08] and parameters: {'sol_per_pop': 72, 'crossover_probability': 0.826652352456685, 'mutation_probability

Number of finished trials:  30


In [7]:

optuna.visualization.plot_pareto_front(study,target_names=["niter","fval"])

In [8]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0], target_name="niter")

In [9]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[1], target_name="fval")

In [10]:
optuna.visualization.plot_parallel_coordinate(study,target=lambda t: t.values[1], target_name="fval")